## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID column 'EIN'.
application_df.drop(columns = ['EIN'], inplace=True)
application_df

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...
34294,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column
for i in application_df.columns:
    print(i, len(application_df[i].unique()))

NAME 19568
APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_counts[app_counts<500].index)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x >1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = list(classification_counts[classification_counts<1000].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df)
application_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_1 DAY RANCH RESCUE AND RURAL OKLAHOMA ANIMAL RESOURCE INC,NAME_100 BLACK MEN OF AMERICA,NAME_100 BLACK MEN OF MEMPHIS INC,NAME_100 BLACK MEN OF WEST GEORGIA INC,NAME_1150 WEBSTER STREET INC,NAME_116TH CAVALRY REGIMENT CHAPTER OF THE US CAVALRY & ARMOR ASSOCIATION,NAME_13TH BOMB SQUADRON ASSOCIATION,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X = application_dummies.drop('IS_SUCCESSFUL', axis=1).values
y = application_dummies['IS_SUCCESSFUL'].values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]
hidden_nodes1=7
hidden_nodes2=14
hidden_nodes3=21

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7)                 137284    
                                                                 
 dense_1 (Dense)             (None, 14)                112       
                                                                 
 dense_2 (Dense)             (None, 21)                315       
                                                                 
 dense_3 (Dense)             (None, 1)                 22        
                                                                 
Total params: 137,733
Trainable params: 137,733
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=200)

Epoch 1/200
684/684 [==============================] - 5s 5ms/step - loss: 0.5805 - accuracy: 0.7239 - val_loss: 0.5921 - val_accuracy: 0.7061
Epoch 2/200
684/684 [==============================] - 3s 4ms/step - loss: 0.3669 - accuracy: 0.8324 - val_loss: 0.4815 - val_accuracy: 0.7898
Epoch 3/200
684/684 [==============================] - 3s 4ms/step - loss: 0.2414 - accuracy: 0.9029 - val_loss: 0.5285 - val_accuracy: 0.7549
Epoch 4/200
684/684 [==============================] - 3s 4ms/step - loss: 0.1694 - accuracy: 0.9358 - val_loss: 0.6505 - val_accuracy: 0.7981
Epoch 5/200
684/684 [==============================] - 3s 4ms/step - loss: 0.1185 - accuracy: 0.9574 - val_loss: 0.6771 - val_accuracy: 0.7992
Epoch 6/200
684/684 [==============================] - 3s 4ms/step - loss: 0.1036 - accuracy: 0.9613 - val_loss: 0.6278 - val_accuracy: 0.8025
Epoch 7/200
684/684 [==============================] - 3s 4ms/step - loss: 0.0981 - accuracy: 0.9625 - val_loss: 0.7478 - val_accuracy: 0.8033

Epoch 58/200
684/684 [==============================] - 2s 4ms/step - loss: 0.1156 - accuracy: 0.9570 - val_loss: 0.6537 - val_accuracy: 0.7105
Epoch 59/200
684/684 [==============================] - 2s 4ms/step - loss: 0.1006 - accuracy: 0.9613 - val_loss: 0.7560 - val_accuracy: 0.7782
Epoch 60/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0928 - accuracy: 0.9641 - val_loss: 1.1138 - val_accuracy: 0.8007
Epoch 61/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0906 - accuracy: 0.9645 - val_loss: 1.3922 - val_accuracy: 0.8036
Epoch 62/200
684/684 [==============================] - 3s 4ms/step - loss: 0.0897 - accuracy: 0.9640 - val_loss: 1.2727 - val_accuracy: 0.8002
Epoch 63/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0893 - accuracy: 0.9643 - val_loss: 1.3534 - val_accuracy: 0.8028
Epoch 64/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0893 - accuracy: 0.9648 - val_loss: 1.8157 - val_accuracy:

Epoch 115/200
684/684 [==============================] - 2s 3ms/step - loss: 0.0852 - accuracy: 0.9656 - val_loss: 4.1568 - val_accuracy: 0.8005
Epoch 116/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0846 - accuracy: 0.9657 - val_loss: 3.7909 - val_accuracy: 0.8025
Epoch 117/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0845 - accuracy: 0.9655 - val_loss: 3.6970 - val_accuracy: 0.7997
Epoch 118/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0845 - accuracy: 0.9655 - val_loss: 4.4151 - val_accuracy: 0.7992
Epoch 119/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0845 - accuracy: 0.9658 - val_loss: 3.5426 - val_accuracy: 0.8010
Epoch 120/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0851 - accuracy: 0.9663 - val_loss: 4.3950 - val_accuracy: 0.7994
Epoch 121/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0848 - accuracy: 0.9666 - val_loss: 4.7468 - val_ac

684/684 [==============================] - 2s 4ms/step - loss: 0.0836 - accuracy: 0.9664 - val_loss: 9.4003 - val_accuracy: 0.7997
Epoch 172/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0831 - accuracy: 0.9662 - val_loss: 9.3536 - val_accuracy: 0.7989
Epoch 173/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0837 - accuracy: 0.9668 - val_loss: 9.7738 - val_accuracy: 0.7992
Epoch 174/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0832 - accuracy: 0.9662 - val_loss: 9.0809 - val_accuracy: 0.7989
Epoch 175/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0828 - accuracy: 0.9665 - val_loss: 9.1785 - val_accuracy: 0.7997
Epoch 176/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0828 - accuracy: 0.9665 - val_loss: 9.3569 - val_accuracy: 0.7992
Epoch 177/200
684/684 [==============================] - 2s 4ms/step - loss: 0.0830 - accuracy: 0.9666 - val_loss: 8.7143 - val_accuracy: 0.7999

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 1.0468 - accuracy: 0.5788 - 579ms/epoch - 2ms/step
Loss: 1.0467650890350342, Accuracy: 0.5787755250930786


In [19]:
# Export our model to HDF5 file
nn.save('../Module 21 Challenge/H5_Files/AlphabetSoupCharity_Optimization1.h5')